### Análise de Dados Epidemiológicos: Dengue, Chikungunya, Zika e Febre Amarela

**Curso:** Bacharelado em Sistemas de Informação  
**Disciplina:** Ciência de Dados 1  
**Equipe:** Lucas André Oliveira Pinheiro, Eduardo Nogueira, Cynthia Neiva  
**Professor:**  Roberto Yuri da Silva Campo  
**Data:** 30/01/2026  


**3️ Importação e Manipulação dos Dados**

Importação dos dados em Python e Exploração com pandas

In [9]:
import pandas as pd
import numpy as np
import os
from pathlib import Path


In [ ]:
data_path = Path("../dataset")

arquivos = list(data_path.glob("*.csv"))

arquivos

In [48]:
colunas_relevantes = [
    'NU_ANO',
    'SG_UF',
    'CS_SEXO',
    'NU_IDADE_N',
    'CS_RACA',
    'CLASSI_FIN',
    'EVOLUCAO',
    'TPAUTOCTO',
    'HOSPITALIZ'
]

dfs = []

for arquivo in arquivos:
    df = pd.read_csv(
        arquivo,
        sep=',',
        encoding='latin1',
        usecols=colunas_relevantes,
        low_memory=True
    )
    dfs.append(df)

dengue = pd.concat(dfs, ignore_index=True)

dengue.head()

,NU_ANO,NU_IDADE_N,CS_SEXO,CS_RACA,SG_UF,HOSPITALIZ,TPAUTOCTO,CLASSI_FIN,EVOLUCAO
0,2015,4038.0,M,4.0,11.0,2.0,NaN,5.0,NaN
1,2015,4026.0,F,4.0,14.0,NaN,NaN,8.0,NaN
2,2015,4020.0,F,4.0,12.0,NaN,NaN,5.0,1.0
3,2015,4017.0,F,4.0,12.0,NaN,NaN,5.0,1.0
4,2015,4031.0,F,3.0,12.0,NaN,NaN,5.0,1.0


In [56]:
# Extração correta da idade em anos

# Unidade
unidade = dengue['NU_IDADE_N'] // 1000

# Idade
idade = dengue['NU_IDADE_N'] % 1000

dengue['IDADE_ANOS'] = np.where(unidade == 4, idade, np.nan)

In [57]:
# Ajuste de tipos para otimização

dengue['NU_ANO'] = dengue['NU_ANO'].astype('int16')

dengue['IDADE_ANOS'] = dengue['IDADE_ANOS'].astype('float32')

In [58]:
# Transformação da variável RAÇA

# Converter para inteiro antes de mapear
dengue['CS_RACA'] = pd.to_numeric(dengue['CS_RACA'], errors='coerce')

# Mapeamento conforme padrão SINAN
mapa_raca = {
    1: "Branca",
    2: "Preta",
    3: "Amarela",
    4: "Parda",
    5: "Indígena",
    9: "Ignorado"
}

dengue['CS_RACA'] = dengue['CS_RACA'].map(mapa_raca)

# Converter para categoria
dengue['CS_RACA'] = dengue['CS_RACA'].astype('category')

In [59]:
# Transformação da variável Estado

# Mapeamento conforme padrão SINAN
mapa_uf = {
    35: "SP", 31: "MG", 52: "GO", 41: "PR", 26: "PE",
    23: "CE", 29: "BA", 50: "MS", 33: "RJ", 53: "DF",
    24: "RN", 51: "MT", 32: "ES", 42: "SC", 25: "PB",
    27: "AL", 17: "TO", 43: "RS", 12: "AC", 15: "PA",
    22: "PI", 13: "AM", 28: "SE", 14: "RR", 16: "AP",
    11: "RO", 21: "MA"
}

# Converter para categoria
dengue['SG_UF'] = dengue['SG_UF'].map(mapa_uf).astype('category')

In [68]:
# Converter para numérico
dengue['TPAUTOCTO'] = pd.to_numeric(dengue['TPAUTOCTO'], errors='coerce')

# Mapear categorias
mapa_zona = {
    1: "Urbana",
    2: "Rural",
    3: "Periurbana",
    9: "Ignorado"
}

dengue['ZONA'] = dengue['TPAUTOCTO'].map(mapa_zona).astype('category')

In [61]:
# Converter demais variáveis categóricas

colunas_categoricas = [
    'CS_SEXO',
    'HOSPITALIZ',
    'CLASSI_FIN',
    'EVOLUCAO'
]

for col in colunas_categoricas:
    dengue[col] = dengue[col].astype('category')

In [62]:
# Verificação final

print("Dimensão final:", dengue.shape)
print("\nInformações do dataset:")
dengue.info(memory_usage='deep')

Dimensão final: (11854752, 11)

Informações do dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11854752 entries, 0 to 11854751
Data columns (total 11 columns):
 #   Column      Dtype   
---  ------      -----   
 0   NU_ANO      int16   
 1   NU_IDADE_N  float64 
 2   CS_SEXO     category
 3   CS_RACA     category
 4   SG_UF       category
 5   HOSPITALIZ  category
 6   TPAUTOCTO   float64 
 7   CLASSI_FIN  category
 8   EVOLUCAO    category
 9   ZONA        category
 10  IDADE_ANOS  float32 
dtypes: category(7), float32(1), float64(2), int16(1)
memory usage: 327.9 MB


In [63]:
dengue['PERIODO'] = np.where(
    dengue['NU_ANO'] < 2020,
    'Pre-Pandemia',
    'Pandemia'
)

dengue['PERIODO'] = dengue['PERIODO'].astype('category')

In [64]:
# Quantidade absoluta
casos_qtd = dengue['PERIODO'].value_counts()

# Percentual
casos_pct = dengue['PERIODO'].value_counts(normalize=True) * 100

casos_qtd_formatado = casos_qtd.apply(lambda x: f"{x:,}".replace(",", "."))

resultado_final = (
    casos_qtd_formatado +
    " (" +
    casos_pct.round(2).astype(str) +
    "%)"
)

print(resultado_final)

PERIODO
Pre-Pandemia    7.961.052 (67.15%)
Pandemia        3.893.700 (32.85%)
dtype: object


In [65]:
# Quantidade absoluta
tabela_qtd = pd.crosstab(
    dengue['PERIODO'],
    dengue['CS_SEXO']
)

# Percentual dentro de cada período
tabela_pct = pd.crosstab(
    dengue['PERIODO'],
    dengue['CS_SEXO'],
    normalize='index'
) * 100


tabela_qtd_formatada = tabela_qtd.applymap(
    lambda x: f"{x:,}".replace(",", ".")
)

tabela_final = (
    tabela_qtd_formatada +
    " (" +
    tabela_pct.round(2).astype(str) +
    "%)"
)

print("Quantidade absoluta e percentual por período e gênero:\n")
print(tabela_final)

Quantidade absoluta e percentual por período e gênero:

CS_SEXO                        F               I                   M
PERIODO                                                             
Pandemia      2.118.495 (54.41%)    3.829 (0.1%)  1.771.339 (45.49%)
Pre-Pandemia  4.397.064 (55.24%)  11.400 (0.14%)  3.552.097 (44.62%)


C:\Users\cynth\AppData\Local\Temp\ipykernel_17008\1427413718.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  tabela_qtd_formatada = tabela_qtd.applymap(


In [66]:
# Quantidade absoluta
tabela_qtd_raca = pd.crosstab(
    dengue['PERIODO'],
    dengue['CS_RACA']
)

# Percentual dentro de cada período
tabela_pct_raca = pd.crosstab(
    dengue['PERIODO'],
    dengue['CS_RACA'],
    normalize='index'
) * 100

tabela_qtd_formatada = tabela_qtd_raca.map(
    lambda x: f"{x:,}".replace(",", ".")
)

tabela_final_raca = (
    tabela_qtd_formatada +
    " (" +
    tabela_pct_raca.round(2).astype(str) +
    "%)"
)

print("Quantidade absoluta e percentual por período e raça:\n")
print(tabela_final_raca)

Quantidade absoluta e percentual por período e raça:

CS_RACA              Amarela              Branca            Ignorado  \
PERIODO                                                                
Pandemia      30.516 (0.78%)  1.594.598 (40.95%)    700.438 (17.99%)   
Pre-Pandemia  52.336 (0.71%)  2.497.389 (33.84%)  2.025.414 (27.45%)   

CS_RACA             Indígena               Parda            Preta  
PERIODO                                                            
Pandemia       9.612 (0.25%)  1.419.959 (36.47%)  138.531 (3.56%)  
Pre-Pandemia  18.406 (0.25%)  2.504.659 (33.94%)  281.087 (3.81%)  


In [ ]:
uf_qtd = dengue['SG_UF'].value_counts().sort_values(ascending=False)

uf_qtd_formatado = uf_qtd.apply(lambda x: f"{x:,}".replace(",", "."))

print("Quantidade de casos por UF:\n")
print(uf_qtd_formatado)

Quantidade de casos por UF:

SG_UF
SP    3.473.298
MG    1.954.331
GO    1.085.877
PR      994.487
PE      512.798
CE      489.389
BA      457.425
MS      337.085
RJ      327.341
DF      243.421
RN      233.192
MT      221.266
ES      218.605
SC      180.889
PB      172.749
AL      140.152
TO      136.339
RS      109.539
AC      107.252
PA       92.108
PI       79.542
AM       75.465
MA       70.576
RO       57.511
SE       50.536
RR       18.896
AP       13.259
Name: count, dtype: object


In [69]:
tabela_qtd_zona = pd.crosstab(dengue['PERIODO'], dengue['TPAUTOCTO'])
tabela_pct_zona = pd.crosstab(
    dengue['PERIODO'],
    dengue['TPAUTOCTO'],
    normalize='index'
) * 100

tabela_final_zona = (
    tabela_qtd_zona.apply(lambda x: x.map(lambda y: f"{y:,}".replace(",", "."))) +
    " (" +
    tabela_pct_zona.round(2).astype(str) +
    "%)"
)

print(tabela_final_zona)

TPAUTOCTO                    1.0              2.0              3.0
PERIODO                                                           
Pandemia         627.344 (93.7%)    21.429 (3.2%)    20.765 (3.1%)
Pre-Pandemia  3.209.388 (91.79%)  124.799 (3.57%)  162.397 (4.64%)
